# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
#Read data from CSV file
weather_df = pd.read_csv("weather_city.csv",index_col=0)
#Print head to verify
weather_df.head()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Rikitea,100,PF,2020-10-25,82,-23.12,-134.97,23.61,10.08
1,Mar del Plata,90,AR,2020-10-25,100,-38.00,-57.56,16.11,1.36
2,Brigantine,90,US,2020-10-25,93,39.41,-74.36,12.78,3.60
3,Tuatapere,96,NZ,2020-10-25,57,-46.13,167.68,12.22,3.13
4,Katsuura,75,JP,2020-10-25,62,35.13,140.30,17.22,3.10


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
#Set api key
gmaps.configure(api_key=g_key)
#Set lat and lng as locations
locations = weather_df[["Lat","Lng"]]
#Set Humidity as weight
weight=weather_df["Humidity"]


In [13]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(weight)

# Create heat layer
heatmap = gmaps.heatmap_layer(
    locations, 
    weights = weight, 
    dissipating=False, 
    max_intensity=100, 
    point_radius=3)

# Add layer
fig.add_layer(heatmap)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
#Narrow hot cities with a max temp over 30°C and over 50% Humidity
hot_cities_df = weather_df.loc[(weather_df["Max Temp"] > 30) & (weather_df["Humidity"] > 50)].dropna()

hot_cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
25,Howard Springs,40,AU,2020-10-25,66,-12.50,131.05,31.11,1.50
77,Ixtapa,75,MX,2020-10-25,66,20.70,-105.20,32.00,3.10
79,Hilo,40,US,2020-10-25,55,19.73,-155.09,31.00,5.10
433,Cabo San Lucas,40,MX,2020-10-25,58,22.89,-109.91,31.11,5.10
443,Lorengau,100,PG,2020-10-25,65,-2.02,147.27,30.06,2.55


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [37]:
#Store into variable named hotel_df.
hotel_df = hot_cities_df
#Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel name"]=""
#Set parameters to search for hotels with 5000 meters.
radius=5000
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel name
25,Howard Springs,40,AU,2020-10-25,66,-12.50,131.05,31.11,1.50,
77,Ixtapa,75,MX,2020-10-25,66,20.70,-105.20,32.00,3.10,
79,Hilo,40,US,2020-10-25,55,19.73,-155.09,31.00,5.10,
433,Cabo San Lucas,40,MX,2020-10-25,58,22.89,-109.91,31.11,5.10,
443,Lorengau,100,PG,2020-10-25,65,-2.02,147.27,30.06,2.55,


In [39]:
# Hit the Google Places API for each city's coordinates.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : radius,
          "key" : g_key}
          
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

In [49]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add location to params 
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Processind record {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    data = response['results']
    
    # Manage the requests into a try/except structure
    try:
        print(f"Closer hotel: {data[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = data[0]['name']

    # Hotels not avaialable
    except (KeyError, IndexError):
        print(f"No hotels found.")


Processind record 25: Howard Springs.
Closer hotel: buffalo beach hostel.
Processind record 77: Ixtapa.
Closer hotel: Flamingo Vallarta Hotel & Marina.
Processind record 79: Hilo.
Closer hotel: Hilo Hawaiian Hotel.
Processind record 433: Cabo San Lucas.
Closer hotel: Waldorf Astoria Los Cabos Pedregal.
Processind record 443: Lorengau.
Closer hotel: Lorengau Harbourside Hotel.


In [50]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel name,Hotel Name
25,Howard Springs,40,AU,2020-10-25,66,-12.50,131.05,31.11,1.50,,buffalo beach hostel
77,Ixtapa,75,MX,2020-10-25,66,20.70,-105.20,32.00,3.10,,Flamingo Vallarta Hotel & Marina
79,Hilo,40,US,2020-10-25,55,19.73,-155.09,31.00,5.10,,Hilo Hawaiian Hotel
433,Cabo San Lucas,40,MX,2020-10-25,58,22.89,-109.91,31.11,5.10,,Waldorf Astoria Los Cabos Pedregal
443,Lorengau,100,PG,2020-10-25,65,-2.02,147.27,30.06,2.55,,Lorengau Harbourside Hotel


In [46]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [47]:

# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))